# [Helm](https://helm.sh/docs/)

Helm é o gerenciador de pacotes do Kubernetes.

Já vimos como criar cada recurso separadamente. O Helm realiza a gestão dos recursos (ns, deploy, svc, ...) desta forma, com apenas um comando, podemos implantar, alterar e excluir todo nosso projeto.

In [ ]:
helm version

Para criar o helm, executamos o comando abaixo:  

```
helm create \<nome-aplicacao>
```

Este comando irá criar a estrutura abaixo:
```
<nome-aplicacao>/
├── Chart.yaml      # Informações Técnicas
├── values.yaml     # Variáveis e Valores
└── templates/      # Modelos de Deployments, Services e todos outros recursos
```

[Helm do Projeto front-app](https://github.com/kdop-dev/front-app/tree/master/helm/front-app)

## Preparando o acesso ao Ambiente

In [5]:
export KUBECONFIG=$PWD/bkp/kubeconfig
mkdir -p work/inovacao
cd $PWD/work/inovacao
echo $PWD

/home/jovyan/work/inovacao


## Projeto front-app

1. Publicando o Projeto front-app

    Para a publicar a aplicação via helm, podemos utilizar duas formas:
    * **helm install** - Comando para publicação da aplicação.
    * **helm upgrade** - Comando para atualização da aplicação. Passando o parâmetro **--install** ele publica a aplicação, caso não exista.

In [6]:
#Informe um nome para o namespace
myNamespace="adsantos"
#helm install --namespace $myNamespace --create-namespace front-app front-app/helm/front-app
helm upgrade --install --namespace $myNamespace --create-namespace front-app front-app/helm/front-app

2. Listando os helms do namespace

In [ ]:
helm list --namespace $myNamespace 

3. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace 

4. Verificando o problema no POD

In [ ]:
kubectl describe pod <insert-pod-name-here> -n $myNamespace 

* O POD necessita da aplicação back-app para poder ser executada.
  * **livenessProbe**: Verifica a saúde do container. Somente se o container estiver em saudável, o POD entrará em execução.  
  Código maior ou igual a 200 e menor que 400 (200 <= StatusCode < 400) indica sucesso. Qualquer outro código indica falha.
```
        livenessProbe:
            httpGet:
              path: /health
              port: http
            initialDelaySeconds: 3 # Tempo para realizar a primeira validação
            periodSeconds: 10      # Tempo em que será realizada a sondagem da atividade.
```
  </br>
  * **readinessProbe**: Indica se o contêiner está pronto para atender às solicitações. Se a análise de prontidão falhar, o controlador de terminais removerá o endereço IP do Pod dos terminais de todos os Serviços que correspondem ao Pod. O estado padrão de prontidão antes do atraso inicial é Failure. Se um contêiner não fornecer uma sonda de prontidão, the default state is Success.


### Como as requisições chegam aos containers?

Existem algumas alternativas para expor nossa aplicação utilizando [serviços](https://kubernetes.io/docs/concepts/services-networking/service/), tais como ClusterIP, LoadBalancer, NodePort, ExternalName. O Ingress não é um serviço é uma configuração para uma aplicação (ingress controller) direcionar o tráfego para o nosso serviço.

![](media/nginx-ingress.png)

Fonte para os ícones: <https://github.com/kubernetes/community/tree/master/icons>

No namespace ingress-nginx está instalado o Ingress Controller, ele é um [Servidor Web Nginx](https://github.com/nginxinc/kubernetes-ingress) modificado e integrado com o kubernetes. Existem outros Ingress Controller, para uma lista mais extensiva acesse [Kubernetes Ingress Controller Overview](https://medium.com/swlh/kubernetes-ingress-controller-overview-81abbaca19ec).

#### Serviços

Como configuramos o nosso serviço? Isso depende de como queremos que ele seja acessado, novamente existem várias opções, as mais comuns são:

* Domínio: front-app.com
* Subdominio: front-app.kdop.net
* Caminho: kdop.net/front-app

Para esta aplicação configuramos a terceira opção. Vamos abrir o arquivo [values.yaml](work/inovacao/front-app/helm/front-app.values.yaml) para entender como foio configurado.

```yaml
service:
  type: ClusterIP
  port: 5000

ingress:
  enabled: true
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$2
  hosts:
    - host: learn.kdop.net
      paths:
      - /adsantos/front-app(/|$)(.*)
  tls: []
```

Esses parâmetros, quando aplicados pelo helm aos templates de serviço e ingress irão criar a seguinte configuração:

```yaml
---
# Source: front-app/templates/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: front-app
  labels:
    helm.sh/chart: front-app-0.1.0
    app.kubernetes.io/name: front-app
    app.kubernetes.io/instance: front-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
spec:
  type: ClusterIP
  ports:
    - port: 5000
      targetPort: http
      protocol: TCP
      name: http
  selector:
    app.kubernetes.io/name: front-app
    app.kubernetes.io/instance: front-app
---
# Source: front-app/templates/ingress.yaml
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: front-app
  labels:
    helm.sh/chart: front-app-0.1.0
    app.kubernetes.io/name: front-app
    app.kubernetes.io/instance: front-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$2
spec:
  rules:
    - host: "learn.kdop.net"
      http:
        paths:
          - path: /adsantos/front-app(/|$)(.*)
            backend:
              serviceName: front-app
              servicePort: 5000
```

> Você pode imprimir o que será criado pelo helm com o comando `helm template` no lugar de `helm install`.

> Sobre TLS ver [README-ingrsss-with-tls.md](README-ingrsss-with-tls.md).

Um serviço do tipo **ClusterIP** foi criado e seu seletor são os PODs com labels `app.kubernetes.io/name` e `app.kubernetes.io/instance` com o valor `front-app`. Não importa quantas réplicas existirem, o serviço irá direcionar o tráfego, na forma de _round-robin_, para cada uma delas.

Mas _ClusterIP_ é um tipo de serviço que só é visível dentro do cluster, pode ser referênciado pelos containers através do seu nome e porta, neste caso `ping front-app` resultaria no retorno do IP associado ao serviço e `telnet front-app 5000` em uma conexão de sucesso.

Para acessar os containers de fora do cluster foi criado uma configuração tipo Ingress, nela foi definido a configuração do frontend que deverá direcionar o tráfego para o backend. Neste cenário, a URL que irá direcionar o tráfego para os nossos containers é http://learn.kdop.net/adsantos/front-app e qualquer caminhos e parâmetros adicionais serão repassados para os nossos containers na forma http://front-app:5000/<o resto da url>.

In [10]:
kubectl describe ingress -n $myNamespace

Name:             cert-app
Namespace:        adsantos
Address:          13.86.33.72
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host            Path  Backends
  ----            ----  --------
  learn.kdop.net  
                  /adsantos/cert-app(/|$)(.*)   cert-app:5000 ()
Annotations:      meta.helm.sh/release-name: cert-app
                  meta.helm.sh/release-namespace: adsantos
                  nginx.ingress.kubernetes.io/rewrite-target: /$2
Events:           <none>


Name:             front-app
Namespace:        adsantos
Address:          13.86.33.72
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host            Path  Backends
  ----            ----  --------
  learn.kdop.net  
                  /adsantos/front-app(/|$)(.*)   front-app:5000 ()
Annotations:      meta.helm.sh/release-name: front-app
                  meta.helm.sh/release-namespace: adsantos
   

Mas para essa configuração funcionar é necessário que um Ingress-Controller esteja instalado e lendo o seu namespace a procura desse tipo de configuração.
    
Neste cluster, o Ingress-Controller instalado é o Nginx, um servidor de web, que funciona como proxy, com um LoadBalancer externo direcionando o tráfego.

In [7]:
kubectl get all -n ingress-nginx

NAME                                            READY   STATUS      RESTARTS   AGE
pod/ingress-nginx-admission-create-85qfk        0/1     Completed   0          3d20h
pod/ingress-nginx-admission-patch-vf2wd         0/1     Completed   0          3d20h
pod/ingress-nginx-controller-5947756d78-4r4f6   1/1     Running     6          3d20h

NAME                                         TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)                      AGE
service/ingress-nginx-controller             LoadBalancer   10.0.99.196    13.86.33.72   80:30238/TCP,443:31738/TCP   3d20h
service/ingress-nginx-controller-admission   ClusterIP      10.0.156.183   <none>        443/TCP                      3d20h

NAME                                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/ingress-nginx-controller   0/1     1            0           3d20h

NAME                                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/ingress-nginx-controller-

O serviço `ingress-nginx-controller` é do tipo **LoadBalancer** e seu endereço externo foi configurado em um DNS (AWS Route 53) para apontar para o seu IP.

In [8]:
kubectl describe service/ingress-nginx-controller -n ingress-nginx

Name:                     ingress-nginx-controller
Namespace:                ingress-nginx
Labels:                   app.kubernetes.io/component=controller
                          app.kubernetes.io/instance=ingress-nginx
                          app.kubernetes.io/managed-by=Helm
                          app.kubernetes.io/name=ingress-nginx
                          app.kubernetes.io/version=0.35.0
                          helm.sh/chart=ingress-nginx-2.13.0
Annotations:              <none>
Selector:                 app.kubernetes.io/component=controller,app.kubernetes.io/instance=ingress-nginx,app.kubernetes.io/name=ingress-nginx
Type:                     LoadBalancer
IP:                       10.0.99.196
LoadBalancer Ingress:     13.86.33.72
Port:                     http  80/TCP
TargetPort:               http/TCP
NodePort:                 http  30238/TCP
Endpoints:                
Port:                     https  443/TCP
TargetPort:               https/TCP
NodePort:              

Aqui você pode obter o IP do Ingress-Controller e a quais portas ele esta associado, neste caso 80 (http) e 443 (https).

## Projeto back-app

1. Publicando o Projeto back-app

In [22]:
helm upgrade --install --namespace $myNamespace --create-namespace back-app back-app/helm/back-app

Release "back-app" has been upgraded. Happy Helming!
NAME: back-app
LAST DEPLOYED: Wed Sep 16 22:22:49 2020
NAMESPACE: adsantos
STATUS: deployed
REVISION: 2
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace adsantos -l "app.kubernetes.io/name=back-app,app.kubernetes.io/instance=back-app" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl --namespace adsantos port-forward $POD_NAME 8080:80


2. Consultando os recursos do namespace

In [23]:
kubectl get all -n $myNamespace

NAME                            READY   STATUS    RESTARTS   AGE
pod/back-app-5789c6585f-6x9bp   1/1     Running   1          26h
pod/back-app-5f4899d7d9-84s96   0/1     Pending   0          10s
pod/cert-app-5b9876fdf8-cfhgf   1/1     Running   1          24h
pod/front-app-8bdf9596c-dxccp   1/1     Running   1          24h

NAME                TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/back-app    ClusterIP   10.0.156.89    <none>        5000/TCP   26h
service/cert-app    ClusterIP   10.0.181.3     <none>        5000/TCP   24h
service/front-app   ClusterIP   10.0.238.137   <none>        5000/TCP   24h

NAME                        READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/back-app    1/1     1            1           26h
deployment.apps/cert-app    1/1     1            1           24h
deployment.apps/front-app   1/1     1            1           24h

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/back-app-5789c6585f   

3. Validando o Serviço

    Para acessar o back-app fora do Cluster, necessitamos expor temporariamente o serviço

In [ ]:
kubectl port-forward svc/back-app 8080:5000 -n $myNamespace 

Consulte o serviço acessando o link abaixo:

http://localhost:8080/health

Agora que os PODs estão em execução, podemos também validar a aplicação front-app.
Verifique no serviço do front-app o IP externo liberado para o loadbalance.

[http://\<ip-loadbalance>:8080/health](http://ip-loadbalance:8080/health)

[http://\<ip-loadbalance>:8080/index](http://ip-loadbalance:8080/index)
    


### Serviços

Nesta aplicação foi criado apenas um serviço do tipo ClusterIP, o que isso significa?

In [13]:
kubectl describe svc/back-app -n $myNamespace

Name:              back-app
Namespace:         adsantos
Labels:            app.kubernetes.io/instance=back-app
                   app.kubernetes.io/managed-by=Helm
                   app.kubernetes.io/name=back-app
                   app.kubernetes.io/version=1.16.0
                   helm.sh/chart=back-app-0.1.0
Annotations:       meta.helm.sh/release-name: back-app
                   meta.helm.sh/release-namespace: adsantos
Selector:          app.kubernetes.io/instance=back-app,app.kubernetes.io/name=back-app
Type:              ClusterIP
IP:                10.0.156.89
Port:              http  5000/TCP
TargetPort:        http/TCP
Endpoints:         
Session Affinity:  None
Events:            <none>


### Volumes

O conceito de volume para PODs é o mesmo que para máquinas VM ou físicas, é uma área de aramzenamento que será montada em um diretório ou em uma letra (windows).

![](media/aks-storage.png)

Fonte: [Opções de armazenamento para aplicativos no Serviço de Kubernetes do Azure (AKS)](https://docs.microsoft.com/pt-br/azure/aks/concepts-storage)

Existem muitos tipos de armazenamentos, os mais comuns são os efêmeros e os persistentes.

* [Efêmeros](https://kubernetes.io/docs/concepts/storage/ephemeral-volumes/): são os mais comuns e são perdidos quando o POD e reiniciado
* [Persistente](https://kubernetes.io/docs/concepts/storage/persistent-volumes/): Como o nome diz, são mantidos mesmo durante reinicio dos PODs. O Kubernetes encontra automaticamente os volumes correspondentes aos PODs.

Consulte [Kubernetes - Volumes](https://kubernetes.io/docs/concepts/storage/), para mais  opções. Além dos tipos, cada sistema oferece tipos diferentes de armazenamentos, tais como SSDs, discos mecânicos, etc. No kubernetes isso é chamado de Storage Class.

Para conhecer as classes de armazenamento disnponíveis no seu cluster, utilize o comando:

In [24]:
kubectl get sc

NAME                PROVISIONER                RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
azurefile           kubernetes.io/azure-file   Delete          Immediate           true                   10d
azurefile-premium   kubernetes.io/azure-file   Delete          Immediate           true                   10d
default (default)   kubernetes.io/azure-disk   Delete          Immediate           true                   26h
managed-premium     kubernetes.io/azure-disk   Delete          Immediate           true                   26h


Os nomes mudam de acordo com o lugar que o seu cluster está instalado, a lista acima são as classes da Azure.

Você pode omitir a classe de aramzenamento, neste caso será utilizada a padrão.

A aplicação acima é uma espécie de banco de dados rudimentar e precisa armazenar seus dados de forma persistente, para tal configuramos um volume persistente.

Vejamos a configuração:

In [ ]:
helm template --namespace adsantos --create-namespace back-app ./back-app

Selecionamos o trecho referente ao volume:

```yaml
---
# Source: back-app/templates/pcv.yaml
kind: PersistentVolumeClaim
metadata:
  name: back-app-managed-disk
spec:
  accessModes:
  - ReadWriteOnce
  storageClassName: managed-premium
  resources:
    requests:
      storage: 5Gi
---
# Source: back-app/templates/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: back-app
  labels:
    helm.sh/chart: back-app-0.1.0
    app.kubernetes.io/name: back-app
    app.kubernetes.io/instance: back-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
spec:
  replicas: 1
  selector:
    matchLabels:
      app.kubernetes.io/name: back-app
      app.kubernetes.io/instance: back-app
  template:
    metadata:
      labels:
        app.kubernetes.io/name: back-app
        app.kubernetes.io/instance: back-app
    spec:
      serviceAccountName: back-app
      securityContext:
        {}
      containers:
        - name: back-app
          securityContext:
            {}
          image: "kdop/back-app:0.0.1"
          imagePullPolicy: Always
          ports:
            - name: http
              containerPort: 5000
              protocol: TCP
          livenessProbe:
            httpGet:
              path: /health
              port: http
          readinessProbe:
            httpGet:
              path: /health
              port: http
          resources:
            limits:
              cpu: 200m
              memory: 256Mi
            requests:
              cpu: 200m
              memory: 256Mi
          volumeMounts:
          - mountPath: "/tmp/inovacao/back-app"
            name: back-volume
      volumes:
        - name: back-volume
          persistentVolumeClaim:
            claimName: back-app-managed-disk
```

Um `PersistentVolume` foi criado dinamicamente em resposta ao nosso `PersistentVolumeClaim`. O K8s interage com as APIs da Azure para criar o PV na forma de um disco.

Você pode criar seus `PersistentVolume` e/ou associá-los a discos já existentes.

In [25]:
kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                            STORAGECLASS      REASON   AGE
pvc-90ee8994-8127-4ba7-af14-92853811d9d8   1Gi        RWO            Delete           Bound    kdop/hub-db-dir                  default                    10d
pvc-fa9ba3f9-d459-4dac-9d5b-d8ac8f1cb6cf   5Gi        RWO            Delete           Bound    adsantos/back-app-managed-disk   managed-premium            18m


O comando `get pv` listou dois `PersistentVolumes`, um de 1Gi e outro de 5Gi, que é o que criamos para nossa aplicação.

Na figura abaixo, você pode ver os dois discos na Azure.

![](media/azure-disks.png)

Se você descrever o `PersistentVolume` de 5Gi, verá alguns labels que descrevem sua origem.

In [29]:
kubectl describe pv/pvc-fa9ba3f9-d459-4dac-9d5b-d8ac8f1cb6cf

Name:              pvc-fa9ba3f9-d459-4dac-9d5b-d8ac8f1cb6cf
Labels:            failure-domain.beta.kubernetes.io/region=centralus
Annotations:       pv.kubernetes.io/bound-by-controller: yes
                   pv.kubernetes.io/provisioned-by: kubernetes.io/azure-disk
                   volumehelper.VolumeDynamicallyCreatedByKey: azure-disk-dynamic-provisioner
Finalizers:        [kubernetes.io/pv-protection]
StorageClass:      managed-premium
Status:            Bound
Claim:             adsantos/back-app-managed-disk
Reclaim Policy:    Delete
Access Modes:      RWO
VolumeMode:        Filesystem
Capacity:          5Gi
Node Affinity:     
  Required Terms:  
    Term 0:        failure-domain.beta.kubernetes.io/region in [centralus]
Message:           
Source:
    Type:         AzureDisk (an Azure Data Disk mount on the host and bind mount to the pod)
    DiskName:     kubernetes-dynamic-pvc-fa9ba3f9-d459-4dac-9d5b-d8ac8f1cb6cf
    DiskURI:      /subscriptions/73c8b399-a4ed-44b8-af3f-081a94

Um olhar mais detalhado no disco na Azure revelará como o Kubernetes faz sua mágica para encontrar seus volumes e seus respectivos donos.

![](media/azure-disk.png)

Descreva o seu `PersistentVolumeClaim` e você encontrará os mesmos labels.

In [30]:
kubectl describe pvc/back-app-managed-disk -n adsantos

Name:          back-app-managed-disk
Namespace:     adsantos
StorageClass:  managed-premium
Status:        Bound
Volume:        pvc-fa9ba3f9-d459-4dac-9d5b-d8ac8f1cb6cf
Labels:        app.kubernetes.io/managed-by=Helm
Annotations:   meta.helm.sh/release-name: back-app
               meta.helm.sh/release-namespace: adsantos
               pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
               volume.beta.kubernetes.io/storage-provisioner: kubernetes.io/azure-disk
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      5Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Mounted By:    back-app-5f4899d7d9-84s96
Events:
  Type    Reason                 Age   From                         Message
  ----    ------                 ----  ----                         -------
  Normal  ProvisioningSucceeded  28m   persistentvolume-controller  Successfully provisioned volume pvc-fa9ba3f9-d459-4dac-9d5b-d8ac8f1cb6cf using kubernetes.io/azure-d

## Projeto cert-app

1. Publicando o Projeto cert-app

In [ ]:
helm upgrade --install --namespace $myNamespace --create-namespace cert-app cert-app/helm/cert-app

2. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace 

3. Validando o Serviço

    Expor o serviço temporariamente para validação

In [ ]:
kubectl port-forward svc/cert-app 8080:5000 -n $myNamespace 

### O tráfego de requisições.

O cert-app foi criado originalmente para ser utilizado apenas pela aplicação front-app, mas foi necessário expó-lo para outras aplicações fora do cluster. Uma vez que ele era coeso e desacoplado da aplicação front-app, a solução era simplesmente criar um endpoint.

Como você resolveria isso? Sabendo que o cert-app já tem um serviço do tipo ClusterIP que é acessado pela front-app no endereço http://cert-app:5000/get-cert?p=MyName.

In [16]:
kubectl get svc/cert-app -n $myNamespace

NAME       TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)    AGE
cert-app   ClusterIP   10.0.181.3   <none>        5000/TCP   22h


Qual o comando para descrever a solução dada?

In [20]:
# digite o comando aqui e ctrl+enter ou command+enter
kubectl describe ingress/cert-app -n $myNamespace

Name:             cert-app
Namespace:        adsantos
Address:          13.86.33.72
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host            Path  Backends
  ----            ----  --------
  learn.kdop.net  
                  /adsantos/cert-app(/|$)(.*)   cert-app:5000 ()
Annotations:      meta.helm.sh/release-name: cert-app
                  meta.helm.sh/release-namespace: adsantos
                  nginx.ingress.kubernetes.io/rewrite-target: /$2
Events:           <none>


Consulte o serviço acessando o link abaixo:

http://localhost:8080/health

## Excluindo as aplicações

In [ ]:
helm delete front-app --namespace $myNamespace  
helm delete back-app --namespace $myNamespace 
helm delete cert-app --namespace $myNamespace 
kubectl delete ns $myNamespace 

[<img align="left" src="media/voltar.png" width="100"/>](02_Kubernetes.ipynb)


ToDo:
* Preparar livenessProbe e readnessProbe
* Preparar ConfigMap e Secret
* Preparar resource request e limits
* Preparar HPA
* comando para o helm listar o que executou